In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import keras


In [22]:
base_model = InceptionV3(input_shape =(512,512,3), include_top = False )

In [23]:
for layer in base_model.layers:
  layer.trainable = False
  

In [24]:
x = Flatten()(base_model.output)
x = Dense(units = 11, activation = 'sigmoid')(x)

model = Model(base_model.input,x)
model.compile(optimizer = 'adam', loss = keras.losses.binary_crossentropy,metrics = ['accuracy'])


In [108]:
train_datagen = ImageDataGenerator(featurewise_center= True,
rotation_range=0.4,
width_shift_range=0.3,
horizontal_flip=True,
preprocessing_function=preprocess_input,
zoom_range=0.4,shear_range=0.4)

train_data = train_datagen.flow_from_directory(directory = 'C:/Users/Kartikay Suri/Downloads/infinity-skncure-master/train',
target_size=(512,512),
batch_size = 64)

test_data = train_datagen.flow_from_directory(directory = 'C:/Users/Kartikay Suri/Downloads/infinity-skncure-master/test',
target_size=(512,512),
batch_size = 64) 

Found 862 images belonging to 11 classes.
Found 427 images belonging to 11 classes.


In [100]:
train_data.class_indices

{'acne': 0,
 'contact dermatitis': 1,
 'corn': 2,
 'cutaneous horn': 3,
 'eczema': 4,
 'herpes zoster': 5,
 'melanoma': 6,
 'normal skin': 7,
 'psoriasis': 8,
 'rosacea': 9,
 'ulticaria': 10}

In [101]:
t_img, label = train_data.next()

In [102]:
t_img.shape

(64, 512, 512, 3)

In [103]:
def plotImages(img_arr, label):
    for idx, img in enumerate(img_arr):
        if idx<=10:
            plt.figure(figsize = (5,5))
            plt.imshow(img)
            plt.title(img.shape)
            plt.axis = False
            plt.show()


In [ ]:
plotImages(t_img,label)

In [105]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
mc = ModelCheckpoint(filepath='./best_model.h5', 
monitor = 'accuracy',
verbose = 1,
save_best_only=True)

# Es = EarlyStopping(monitor="accuracy",
# min_delta = 0.01, 
# patience=5, 
# verbose=1)

# cb = [mc,Es]
cb = [mc]

In [ ]:
his = model.fit_generator(train_data, 
steps_per_epoch = 10, 
epochs = 20, callbacks = cb)

In [ ]:
path = r'C:\Users\Kartikay Suri\Downloads\infinity-skncure-master\test\corn\18.png'
img = load_img(path,target_size = (512,512))
i = img_to_array(img)
i = preprocess_input(i)
input_arr = np.array([i])
input_arr.shape
pred = np.argmax(model.predict(input_arr))
print(pred)

plt.imshow(input_arr[0])
plt.title("input image")
plt.show()


In [ ]:
import os
newpath = os.listdir(r'C:\Users\Kartikay Suri\Downloads\infinity-skncure-master\test')
for k in newpath:
    subpath = os.listdir('C:/Users/Kartikay Suri/Downloads/infinity-skncure-master/test/'+k)
    for j in subpath:
        checkpath = 'C:/Users/Kartikay Suri/Downloads/infinity-skncure-master/test/'+ str(k) + '/' + str(j)
        print(checkpath)
        img = load_img(checkpath,target_size = (512,512))
        i = img_to_array(img)
        i = preprocess_input(i)
        input_arr = np.array([i])
        input_arr.shape
        pred = np.argmax(model.predict(input_arr))
    

In [8]:
model = load_model('C:/Users/Kartikay Suri/Documents/Code Directory/Python program/pocketdermaNew3.h5')


In [12]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
import datetime
import os
import requests
from keras.models import load_model

# # Fetch the service account key JSON file contents
# cred = credentials.Certificate("C:/Users/Kartikay Suri/Downloads/pocket-derm-66975-firebase-adminsdk-qima1-21949fb43c.json")

# # Initialize the app with a service account, granting admin privileges
# app = firebase_admin.initialize_app(cred, {
#     'storageBucket': 'pocket-derm-66975.appspot.com',
# }, name='storage')

bucket = storage.bucket(app=app)
blob = bucket.blob("image1.jpeg")

print(blob.generate_signed_url(datetime.timedelta(seconds=300), method='GET'))
response = requests.get(blob.generate_signed_url(datetime.timedelta(seconds=300), method='GET'))

file = open("sample_image.png", "wb")
file.write(response.content)
file.close()


checkpath = 'C:/Users/Kartikay Suri/Documents/Code Directory/Python program/sample_image.png'
print(checkpath)
img = load_img(checkpath,target_size = (224,224))
i = img_to_array(img)
i = preprocess_input(i)
input_arr = np.array([i])
input_arr.shape
pred = np.argmax(model.predict(input_arr))
print(pred)
classes = {1:"Acne",2: "Corn",3: "Cutaneous Horn",4: "Measles",5:"Melanoma",6:"Psoriasis",7:"Ringworm"}
result_class = classes[pred]
print(result_class)
if pred == 0:
    info = """Acne is a skin condition that occurs when your hair follicles become 
    plugged with oil and dead skin cells. It causes whiteheads, blackheads or pimples. 
    Acne is most common among teenagers, though it affects people of all ages.

    Effective acne treatments are available, but acne can be persistent. 
    The pimples and bumps heal slowly, and when one begins to go away, others seem to crop up.

    Depending on its severity, acne can cause emotional distress and scar the 
    skin. The earlier you start treatment, the lower your risk of such problems."""

    Website = "https://www.mayoclinic.org/diseases-conditions/acne/symptoms-causes/syc-20368047"

elif pred == 1:
    info = """Corns and calluses are thick, hardened layers of skin that develop when your skin 
    tries to protect itself against friction and pressure. 
    They most often develop on the feet and toes or hands and fingers. Corns and calluses can be unsightly.

    If you're healthy, you need treatment for corns and calluses only if they cause discomfort. 
    For most people, simply eliminating the source of friction or pressure makes corns and calluses disappear.

    If you have diabetes or another condition that causes poor blood flow to your feet, 
    you're at greater risk of complications from corns and calluses. 
    Seek your doctor's advice on proper care for corns and calluses if you have such a condition."""

    Website = """https://www.mayoclinic.org/diseases-conditions/corns-and-calluses/symptoms-causes/syc-20355946#:~:text=Overview,and%20calluses%20can%20be%20unsightly."""
elif pred == 2:
    info = """A cutaneous horn (cornu cutaneum) is a hard conical projection from the skin, made of compact keratin. 
    They are so named as they resemble an animal's horn. They arise from benign, premalignant or malignant skin lesions.
    Around half of horns have a benign base, and half are premalignant or malignant. 
    The most common underlying lesions are seborrhoeic keratosis, viral warts (due to human papillomavirus), 
    actinic keratosis, and well-differentiated squamous cell carcinoma (associated with exposure 
    to the sun and other sources of UV radiation)."""

    Website = """https://dermnetnz.org/topics/cutaneous-horn"""

elif pred ==3:
    info = """Measles is a childhood infection caused by a virus. Once quite common, 
    measles can now almost always be prevented with a vaccine.

    Also called rubeola, measles can be serious and even fatal for small children. 
    While death rates have been falling worldwide as more children receive the measles vaccine,
     the disease still kills more than 100,000 people a year, most under the age of 5."""

elif pred == 4:
    info = """"""

elif pred == 5:
    info = """"""

elif pred ==6:
    info = """"""


https://storage.googleapis.com/pocket-derm-66975.appspot.com/image1.jpeg?Expires=1642927852&GoogleAccessId=firebase-adminsdk-qima1%40pocket-derm-66975.iam.gserviceaccount.com&Signature=fpMuGVdAObIbsk1jaXRn%2Blwv7SpqumHscpDCWYYyWM%2FVIXVxZmUocJR4k48uZgVqg8ZqEIbbC2jfVw0l9tkEfFPAk9dagro69y1zsulwf68i6tE%2FJ0s0wGoHo0jfXGuOFscnwa2Q7Osmf5YzGsIKM%2Fq3QPbVgAxD0gO0YW8x485RyIgDlP%2B6hCkASiNOrDBXYW7ntKHXCirooilUDB15J%2BhRoaXw4adYIN%2B67jNC3zK%2FmWPtc14oPt2CvsjNp%2F1igK2vFvqzRF5Q7tx4uNuHdm4X9OIGgJxqJ%2Fv02s%2Bva2IYQdq7nOu5sYnh1cWKFLbaxPsSRNrtqOiwEMYEJHMsgw%3D%3D
C:/Users/Kartikay Suri/Documents/Code Directory/Python program/image1.jpg
4
Measles


In [2]:
from firebase import firebase  
firebase = firebase.FirebaseApplication('https://pocket-derm-66975-default-rtdb.firebaseio.com/', None)  
data =  { 'Name': 'Test'}  
result = firebase.post('/Test/',data)  
print(result)  

{'name': '-Mu52q89dVtF_meYMTcs'}


C:/Users/Kartikay Suri/Documents/Code Directory./Python program/sample_image.png
1
